# Text Analytics

El Procesamiento del lenguaje natural (NLP) es una parte de la inteligencia artificial que engloba el lenguaje escrito y hablado. Puede usar NLP para crear soluciones que extraigan el significado semántico de instrucciones habladas o escritas, o que formulen respuestas adecuadas en lenguaje natural.

Microsoft Azure *Cognitive Services* incluye el servicio *Text Analytics*, que cuenta con algunas funciones de NLP integradas, como la identificación de frases clave en texto y la clasificación de texto según opiniones.

![Un robot leyendo un cuaderno](./images/NLP.jpg)

Por ejemplo, supongamos que la organización ficticia *Margie’s Travel* solicita a los clientes que den su opinión sobre sus estancias en los hoteles. Puede usar el servicio Text Analytics para resumir las reseñas con sus frases clave, determinar qué reseñas son positivas y cuáles negativas o analizar el texto de la reseña en busca de menciones de entidades conocidas, como las ubicaciones o las personas.

## Ver los documentos de reseñas

Empezaremos echando un vistazo a algunas reseñas de clientes sobre hoteles.

Las reseñas se incluyen en archivos de texto. Para verlas, haga clic en **Run cell** (&#9655;) a la izquierda de la celda y ejecute el código siguiente.

In [ ]:
import os

# Leer las reseñas en la carpeta /data/reviews
reviews_folder = os.path.join('data', 'text', 'reviews')

# Crear una recopilación de reseñas con ID (nombre de archivo) y propiedades de texto (contenidos)
reviews = []
for file_name in os.listdir(reviews_folder):
    review_text = open(os.path.join(reviews_folder, file_name)).read()
    review = {"id": file_name, "text": review_text}
    reviews.append(review)

for review_num in range(len(reviews)):
    # imprima el texto de reseña
    print('{}\n{}\n'.format(reviews[review_num]['id'], reviews[review_num]['text']))

## Crear un recurso de Cognitive Services

Para analizar el texto en estas reseñas, puede usar el servicio **Text Analytics**, de Cognitive Services. Para ello, debe aprovisionar un recurso de **Text Analytics** o de **Cognitive Services** en su suscripción de Azure (utilice un recurso de Text Analytics si es el único servicio que piensa usar o si quiere supervisar su uso de forma independiente. Si no es así, puede usar un recurso de Cognitive Services para combinar el servicio Text Analytics y otros servicios, lo que permite que los desarrolladores utilicen un mismo punto de conexión y clave para acceder a los recursos).

Si no tiene uno, siga estos pasos para crear un recurso de **Cognitive Services** en su suscripción de Azure:

> **Nota**: Si ya tiene un recurso de Cognitive Services, abra su página de **Inicio rápido** en Azure Portal y copie la clave y el punto de conexión en la siguiente celda. En caso contrario, siga estos pasos para crear uno.

1. En la pestaña de otro explorador, abra Azure Portal (https://portal.azure.com) e inicie sesión con su cuenta de Microsoft.
2. Haga clic en el botón **&#65291;Crear un recurso**, busque *Cognitive Services* y cree un recurso de **Cognitive Services** con esta configuración:
    - **Suscripción**: *su suscripción de Azure*.
    - **Grupo de recursos**: *seleccione o cree un grupo de recursos con un nombre único.*
    - **Región**: *seleccione cualquier región disponible*:
    - **Nombre**: *escriba un nombre único*.
    - **Plan de tarifa**: S0
    - **Confirmo que he leído y comprendido las notificaciones**: seleccionado.
3. Espere a que la implementación finalice. Vaya al recurso de Cognitive Services y, en la página **Información general**, haga clic en el vínculo para administrar las claves del servicio. Necesitará el punto de conexión y las claves para conectarse a su recurso de Cognitive Services desde aplicaciones de cliente.

### Obtener la clave y el punto de conexión de un recurso de Cognitive Services

Para usar su recurso de Cognitive Services, las aplicaciones de cliente necesitan su clave de autenticación y su punto de conexión:

1. En Azure Portal, en la página **Claves y punto de conexión** de su recurso de Cognitive Services, copie la **Key1** de su recurso y péguela en el siguiente código, en sustitución de **YOUR_COG_KEY**.
2. Copie el **Punto de conexión** de su recurso y péguelo en el siguiente código, en sustitución de **YOUR_COG_ENDPOINT**.
3. Haga clic en el botón verde <span style="color:green">&#9655;</span> para ejecutar el código de la siguiente celda.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

## Detectar idioma
Empezaremos por identificar el idioma en el que se han escrito las reseñas.

In [ ]:
import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Obtener un cliente para el recurso Text Analytics de Cognitive Services
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))

# Analizar las reseñas de la carpeta /data/reviews que ya ha leído
language_analysis = text_analytics_client.detect_language(documents=reviews)

# Imprimir los detalles de detección de idioma de cada reseña
for review_num in range(len(reviews)):
    # imprima el id. de reseña
    print(reviews[review_num]['id'])

    # Obtenga los detalles de idioma para esta reseña
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(' - Language: {}\n - Code: {}\n - Score: {}\n'.format(lang.name, lang.iso6391_name, lang.score))

    # Agregue el código de idioma detectado para la colección de reseñas (para mayor análisis)
    reviews[review_num]["language"] = lang.iso6391_name

## Extraer frases clave

Ahora, puede analizar el texto de las reseñas de clientes para identificar frases clave que incluyan información sobre los puntos clave.

In [ ]:
# # Usar el cliente y las reseñas creadas en la celda de código anterior para obtener frases clave
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)

# Imprimir las frases clave de cada reseña
for review_num in range(len(reviews)):
    # imprima el id. de reseña
    print(reviews[review_num]['id'])

    # Obtenga las frases clave de esta reseña
    print('\nKey Phrases:')
    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    # Imprima cada frase clave
    for key_phrase in key_phrases:
        print('\t', key_phrase)
    print('\n')

Las frases clave pueden ayudar a comprender mejor los puntos clave de cada reseña. Por ejemplo, una reseña que contenga la frase “buen personal” o “mal servicio” puede otorgar información sobre la experiencia del cliente.

## Determinar la opinión

Puede ser útil clasificar las reseñas como *positivas* o *negativas* mediante una *puntuación de opinión*. Para ello puede usar el servicio Text Analytics.

In [ ]:
# Utilizar el cliente y las reseñas creadas para obtener puntuaciones de opiniones
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

# Imprimir los resultados de cada reseña
for review_num in range(len(reviews)):

    # Obtenga la puntuación de opinión para esta reseña
    sentiment_score = sentiment_analysis.documents[review_num].score

    # Clasifique como 'positiva' si es más de 0,5, 
    if sentiment_score < 0.5:
        sentiment = 'negative'
    else:
        sentiment = 'positive'

    # imprima el nombre de archivo y la opinión
    print('{} : {} ({})'.format(reviews[review_num]['id'], sentiment, sentiment_score))

## Extraer entidades conocidas

Las *entidades* son elementos mencionados en el texto que hacen referencia a otro tipo de elemento conocido. Por ejemplo: una ubicación, una persona o una fecha. Si estuviera interesado en conocer las fechas y lugares mencionados en las reseñas podría usar el siguiente código para encontrarlos.

In [ ]:
# Use el cliente y las reseñas creadas para obtener entidades con nombre
entity_analysis = text_analytics_client.entities(documents=reviews)

# Imprimir los resultados de cada reseña
for review_num in range(len(reviews)):
    print(reviews[review_num]['id'])
    # Obtenga las entidades con nombre de esta reseña
    entities = entity_analysis.documents[review_num].entities
    for entity in entities:
        # Solo obtenga las entidades de ubicación
        if entity.type in ['DateTime','Location']:
            link = '(' + entity.wikipedia_url + ')' if entity.wikipedia_id is not None else ''
            print(' - {}: {} {}'.format(entity.type, entity.name, link))

Tenga en cuenta que algunas entidades son lo suficientemente conocidas como para tener una página de Wikipedia. En estos casos, el servicio Text Analytics devuelve la URL de esa página.

## Más información

Para obtener más información sobre el servicio Text Analytics, consulte la [documentación del servicio Text Analytics](https://docs.microsoft.com/azure/cognitive-services/text-analytics/)